MAIN CODE start with STL FILE and Force Application.

possible improvments:
- implementing pressure.
- more dofs

- visualize Von mises


In [ ]:
### GOOD ONE

import numpy as np
from skfem import *
import pytetwild
import meshio
from skfem.models.elasticity import linear_stress
from skfem.helpers import grad, transpose
from skfem.visuals.matplotlib import draw
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

transverse_modulus = 2432.29e6 #Pa
principal_modulus = 2367.10e6 # Pa

transvere_poisson = 0.24
principal_poisson = 0.34

def TetMesh(input_file, output_file):
    stl: meshio.Mesh = meshio.read(f"C:/Users/AlexP/Desktop/BA/STL_Files/{input_file}")
    vertices, tetrahedras = pytetwild.tetrahedralize(stl.points, stl.cells_dict["triangle"])

    cells = meshio.CellBlock('tetra', tetrahedras)
    # in case you want to directly continue with it
    p = np.array([vertices[:, 0], vertices[:, 1], vertices[:, 2]], dtype=np.float64)
    t = np.array(tetrahedras, dtype=np.float64).T
    m = MeshTet(p, t)
    return m

def transverse_isotropy(E1, E2, nu1, nu2):
    # E1->elasticity modulus in the isotropic plane
    # E2->e.m. in transverse axis
    C11 = E1 * (1 - nu2 ** 2) / (1 - nu1 ** 2)
    C12 = E1 * nu1 / (1 - nu1 ** 2)
    C13 = E2 * nu2 / (1 - nu2 ** 2)
    C33 = E2 / (1 - nu2 ** 2)
    C44 = E1 / (2 * (1 + nu1))
    C66 = 0.5 * (C11 - C12)
    return C11, C12, C13, C33, C44, C66

C11, C12, C13, C33, C44, C66 = transverse_isotropy(transverse_modulus, principal_modulus, transvere_poisson, principal_poisson)

def Stiffnesstensor(C11, C12, C13, C33, C44, C66):
    C = np.zeros((3, 3, 3, 3))
    C[0, 0, 0, 0] = C11
    C[1, 1, 1, 1] = C11
    C[0, 0, 1, 1] = C12
    C[1, 1, 0, 0] = C12
    C[0, 0, 2, 2] = C13
    C[2, 2, 0, 0] = C13
    C[1, 1, 2, 2] = C13
    C[2, 2, 1, 1] = C13
    C[2, 2, 2, 2] = C33
    C[1, 2, 1, 2] = C44
    C[2, 1, 2, 1] = C44
    C[2, 1, 1, 2] = C44
    C[1, 2, 2, 1] = C44
    C[0, 2, 0, 2] = C44
    C[2, 0, 2, 0] = C44
    C[2, 0, 0, 2] = C44
    C[0, 2, 2, 0] = C44
    C[0, 1, 0, 1] = C66
    C[1, 0, 1, 0] = C66
    C[1, 0, 0, 1] = C66
    C[0, 1, 1, 0] = C66
    return C 

C = Stiffnesstensor(C11, C12, C13, C33, C44, C66)

def calculate_von_mises_stress(stress_tensor):
    """
    Calculate the von Mises stress from the stress tensor.

    Parameters:
    stress_tensor (np.ndarray): Stress tensor of shape (3, 3, n), where n is the number of points/nodes.

    Returns:
    np.ndarray: Von Mises stress of shape (n).
    """
    sigma_xx = stress_tensor[0, 0]
    sigma_yy = stress_tensor[1, 1]
    sigma_zz = stress_tensor[2, 2]
    sigma_xy = stress_tensor[0, 1]
    sigma_yz = stress_tensor[1, 2]
    sigma_zx = stress_tensor[2, 0]

    von_mises_stress = np.sqrt(
        0.5 * ((sigma_xx - sigma_yy) ** 2 +
               (sigma_yy - sigma_zz) ** 2 +
               (sigma_zz - sigma_xx) ** 2 +
               6.0 * (sigma_xy ** 2 + sigma_yz ** 2 + sigma_zx ** 2))
    )

    return von_mises_stress

def yield_stress(array, x):
    e = []
    p = []
    for i in range(len(array)):
        if array[i] > x:
            p.append(i)
        
    return {'plastic': p}

def fem_in_directions(stl_input, app_force, yield_stress_value):
    stl = meshio.read(f"C:/Users/AlexP/Desktop/BA/STL_Files/{stl_input}")
    vertices, tetrahedras = pytetwild.tetrahedralize(stl.points, stl.cells_dict["triangle"])

    cells = meshio.CellBlock('tetra', tetrahedras)
    p = np.array([vertices[:, 0], vertices[:, 1], vertices[:, 2]], dtype=np.float64)
    t = np.array(tetrahedras, dtype=np.float64).T
    m = MeshTet(p, t)

    # Define elements
    e1 = ElementTetP1()
    e = ElementVector(e1)
    ib = Basis(m, e, MappingIsoparametric(m, e1), 3)
    basis0 = Basis(m, e1)
    @BilinearForm
    def anisotropic(u, v, _):
        epsu = 0.5 * (grad(u) + transpose(grad(u)))
        epsv = 0.5 * (grad(v) + transpose(grad(v)))
        return np.einsum('ijkl,ij...,kl...', C, epsu, epsv)

    # Assemble stiffness matrix
    K = asm(anisotropic, ib)

    # Define DOFs
    dofs = {
        'left': ib.get_dofs(lambda x: np.isclose(x[0], x[0].min())),
        'right': ib.get_dofs(lambda x: np.isclose(x[0], x[0].max())),
        'up': ib.get_dofs(lambda x: np.isclose(x[1], x[1].max())),
        'down': ib.get_dofs(lambda x: np.isclose(x[1], x[1].min())),
        'front': ib.get_dofs(lambda x: np.isclose(x[2], x[2].max())),
        'back': ib.get_dofs(lambda x: np.isclose(x[2], x[2].min()))
    }

    # Define displacements
    displacements = {
        'left': ('right', 'u^1', app_force),
        'right': ('left', 'u^1', -app_force),
        'up': ('down', 'u^2', -app_force),
        'down': ('up', 'u^2', app_force),
        'front': ('back', 'u^3', -app_force),
        'back': ('front', 'u^3', app_force)
    }

    results = {}
    for direction, (opposite, component, force_value) in displacements.items():
        u = ib.zeros()

        # Initialize force array
        F = np.zeros(u.shape)

        # Correctly index the nodal DOFs
        direction_dofs = dofs[direction].nodal[component].astype(int).flatten()
        F[direction_dofs] = force_value

        # Fix boundary conditions
        fixed_dofs = np.hstack([dofs[opposite].all()])
        u[fixed_dofs] = 0

        # Set DOFs and solve the system
        free_dofs = np.setdiff1d(np.arange(K.shape[0]), fixed_dofs)
        K_free = K[free_dofs][:, free_dofs]
        F_free = F[free_dofs]
        u_free = solve(K_free, F_free)
        u[free_dofs] = u_free
        # compute the strain tensor
         # Calculate final shift
        final_shift = u[direction_dofs]

        # Implement shifts
        u[direction_dofs] = final_shift

        # Flatten the DOFs to fix the issue
        I = ib.complement_dofs(np.concatenate([dofs[direction], dofs[opposite]]))
        u = solve(*condense(K, x=u, I=I))
        sf = 1
        # Translate and save the mesh
        m_shifted = m.translated(sf * u[ib.nodal_dofs])
        m_shifted.save(f"C:/Users/AlexP/Desktop/BA/Meshes/TESTS_{stl_input}_{direction}.vtk", {"affected": u[ib.nodal_dofs][0]})
        
        
        
        
        
        strain = ib.interpolate(u)
        strain_tensor = np.array([
            [strain.grad[0][0], 0.5*(strain.grad[0][1] + strain.grad[1][0]), 0.5*(strain.grad[0][2] + strain.grad[2][0])],
            [0.5*(strain.grad[1][0] + strain.grad[0][1]), strain.grad[1][1], 0.5*(strain.grad[1][2] + strain.grad[2][1])],
            [0.5*(strain.grad[2][0] + strain.grad[0][2]), 0.5*(strain.grad[2][1] + strain.grad[1][2]), strain.grad[2][2]]
        ])

        # Reshape strain_tensor to match C dimensions
        strain_tensor_reshaped = strain_tensor.reshape(3, 3, -1)

        # Compute the stress tensor using the anisotropic constitutive matrix C
        stress_tensor = np.einsum('ijkl,klm->ijm', C, strain_tensor_reshaped)

        # Calculate von Mises stress (element-wise)
        von_mises_stress_elem = calculate_von_mises_stress(stress_tensor)

        # Interpolate element-wise von Mises stress to nodes
        nodal_values = np.zeros(m.p.shape[1])
        element_counts = np.zeros(m.p.shape[1])
        for i in range(m.t.shape[1]):
            nodal_values[m.t[:, i]] += von_mises_stress_elem[i]
            element_counts[m.t[:, i]] += 1
        von_mises_stress_nodal = nodal_values / element_counts

        # Check yield stress
        yield_results = yield_stress(von_mises_stress_nodal, yield_stress_value)

        # Save the results to a VTK file
        #m.save(f"C:/Users/AlexP/Desktop/BA/Meshes/TESTS{stl_input}_{direction}.vtk", 
        #       point_data={"von_mises_stress": von_mises_stress_nodal})

        results[direction] = yield_results

    return results


# Example usage
Force = 1e9  # Applied force
stl_input = "20mm_cube.stl"
yield_stress_value = 2e9  # Yield stress value

results = fem_in_directions(stl_input, Force, yield_stress_value)
print(results)
